In [1]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Convolution2D, Dropout, MaxPool2D, AvgPool2D, \
Concatenate, Flatten, GlobalAveragePooling2D
from keras import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from attention import Attention
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [2]:
input1 = Input(shape=(48, 48*100,))
input2 = Input(shape=(8,100,))
input2_ = Flatten()(input2)
input3 = Input(shape=(299,299,3))

base_model = InceptionV3(weights='imagenet', include_top=False)
outputs = []
for i in range(7):
#     att = Attention(48)(input1)
#     conv = Convolution2D(kernel_size=(3,3), filters=16, padding='valid')(input1)
#     conv = MaxPool2D()(conv)
#     conv = Convolution2D(kernel_size=(3,3), filters=16, padding='valid')(conv)
#     conv = MaxPool2D()(conv)
#     conv = Convolution2D(kernel_size=(5,5), filters=16, padding='valid')(conv)
#     conv = MaxPool2D()(conv)
#     conv = MaxPool2D()(conv)
#     max_pool = Flatten()(conv)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    concat = Concatenate()([input2_, x])
    fc = Dense(64, activation='relu')(concat)
    dp = Dropout(0.3)(fc)
    outputs.append(Dense(8, activation='relu')(dp))

In [3]:
model = Model(inputs=[input2, base_model.input], outputs=outputs)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [4]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=[metrics.MSE])

In [5]:
X1 = np.load('X1.npy')
X2 = np.load('X2.npy')
X3 = np.load('X3.npy')
y = np.load('y.npy')

In [6]:
X1 = np.mean(X1, axis=3)
# X2 = np.mean(X2, axis=3)

In [7]:
train_size = int(X3.shape[0]*0.8)
x1_train, x1_test = X1[:train_size, :, :], X1[train_size:, :, :]
x2_train, x2_test = X2[:train_size, :, :], X2[train_size:, :, :]
x3_train, x3_test = X3[:train_size, :, :], X3[train_size:, :, :]
y_train, y_test = y[:train_size, :, :], y[train_size:, :, :]
# x_train, x_test, y_train, y_test = train_test_split([X1, X2], y, test_size=0.2)
y_train = np.einsum('jik->ijk', y_train)
y_test = np.einsum('jik->ijk', y_test)

In [ ]:
model.fit([x2_train, x3_train], [y_train[i] for i in range(7)],
          validation_data=[[x2_test,x3_test], [y_test[i] for i in range(7)]], 
          epochs=20, batch_size=16)

Train on 558 samples, validate on 140 samples
Epoch 1/20
558/558 [==============================] - 20s 35ms/step - loss: 7681145.5430 - dense_1_loss: 563964.3930 - dense_3_loss: 1451683.6537 - dense_5_loss: 256673.0533 - dense_7_loss: 1537301.1779 - dense_9_loss: 1532856.8687 - dense_11_loss: 1249382.4763 - dense_13_loss: 1089284.0258 - dense_1_mean_squared_error: 563964.3930 - dense_3_mean_squared_error: 1451683.6537 - dense_5_mean_squared_error: 256673.0533 - dense_7_mean_squared_error: 1537301.1779 - dense_9_mean_squared_error: 1532856.8687 - dense_11_mean_squared_error: 1249382.4763 - dense_13_mean_squared_error: 1089284.0258 - val_loss: 8972252.1714 - val_dense_1_loss: 671008.2054 - val_dense_3_loss: 1238513.0964 - val_dense_5_loss: 332897.7893 - val_dense_7_loss: 1504766.8661 - val_dense_9_loss: 1175292.3500 - val_dense_11_loss: 1663665.6768 - val_dense_13_loss: 2386108.2571 - val_dense_1_mean_squared_error: 671008.2054 - val_dense_3_mean_squared_error: 1238513.0964 - val_dense_